# Load Data

In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import lxml

2024-11-02 01:18:21.293240: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-02 01:18:21.297654: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-02 01:18:21.324833: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1730510301.480628   30529 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1730510301.499890   30529 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-02 01:18:21.558158: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [2]:
LOWER_MEKONG_STATION_CODES =  [
    "STR", # StungTreng
    "KRA", # Kratie
    "KOM", # Kompong Cham
    "PPB", # Phnom Penh (Bassac)
    "PPP", # Phnom Penh Port
    "KOH", # Koh Khel (Bassac)
    "NEA", # Neak Luong
    "PRE", # Prek Kdam (Tonle Sap)
    "TCH", # Tan Chau
    "CDO", # Chau Doc (Bassac)
]
BASE_URL = "http://ffw.mrcmekong.org/fetchwet_st.php?StCode="
r = requests.get(BASE_URL+LOWER_MEKONG_STATION_CODES[3], verify=False)
# soup = BeautifulSoup(r.content, 'html5lib')
# body = soup.find('body')
data_string = r.content.decode('utf-8')

# Convert single quotes and remove any non-JSON parts
data_string = data_string.replace('date_gmt:', '"date_gmt":')
data_string = data_string.replace('Max:', '"Max":')
data_string = data_string.replace('Min:', '"Min":')
data_string = data_string.replace('AVG:', '"AVG":')
data_string = data_string.replace('floodLevel:', '"floodLevel":')
data_string = data_string.replace('alarmLevel:', '"alarmLevel":')
for year in range(1992, 2025):
    data_string = data_string.replace(f'{year}:', f'"{year}":')

data_string = data_string.replace(',]', ']')

# Now parse it into a list of dictionaries
data = json.loads(data_string)

# Convert to dataframe
df = pd.DataFrame(data)
df['date_gmt'] = df['date_gmt'].apply(lambda x: x.split("-")[1]+"-"+x.split("-")[2])
df['station'] = LOWER_MEKONG_STATION_CODES[3]

# Set date_gmt as index 
df.index = df['date_gmt']

df.describe()

/home/vscode/.local/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ffw.mrcmekong.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


,Max,Min,AVG,1992,1998,2000,2016,2017,2018,2019,2020,2021,2022,2023,2024,floodLevel,alarmLevel
count,153.000000,153.0,153.000000,153.000000,153.000000,153.000000,153.000000,153.000000,153.000000,153.000000,153.000000,153.000000,153.000000,153.000000,153.000000,153.0,153.0
mean,9.479281,0.0,7.140654,6.269608,5.489020,9.092484,5.459869,7.036275,7.262484,4.709412,4.009477,4.709281,6.199608,5.300261,5.333987,12.0,10.5
std,1.792758,0.0,2.184002,2.262871,1.771895,1.952256,2.049587,1.856611,2.555734,2.435415,1.792308,1.689127,2.031904,2.354565,2.616292,0.0,0.0
min,4.900000,0.0,2.690000,2.040000,2.420000,4.700000,1.940000,3.550000,2.590000,1.990000,1.640000,1.760000,3.240000,0.000000,0.000000,12.0,10.5
25%,8.240000,0.0,5.290000,4.400000,4.760000,7.620000,4.150000,4.670000,4.190000,2.420000,2.270000,3.220000,3.840000,2.670000,2.750000,12.0,10.5
50%,10.390000,0.0,8.140000,6.910000,5.810000,9.760000,5.640000,8.130000,8.140000,4.070000,4.170000,4.730000,6.790000,6.280000,6.370000,12.0,10.5
75%,10.770000,0.0,9.000000,8.340000,6.870000,10.490000,7.420000,8.440000,9.600000,6.630000,5.360000,6.380000,7.840000,7.430000,7.160000,12.0,10.5
max,11.200000,0.0,9.450000,9.010000,7.920000,11.200000,8.130000,8.860000,9.950000,9.490000,7.700000,7.560000,9.220000,8.260000,9.100000,12.0,10.5


# Preprocess

In [3]:
df_filtered = df[['date_gmt', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024']]
df_filtered.set_index('date_gmt', inplace=True)
df_filtered.reset_index(inplace=True)
df_long = pd.melt(df_filtered, id_vars=['date_gmt'], value_vars=['2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024'],
                   var_name='Year', value_name='water_level')
df_long['DATE_GMT'] = pd.to_datetime(df_long['date_gmt'] + '-' + df_long['Year'], format='%m-%d-%Y').dt.strftime('%m-%d-%Y')
df_long = df_long[['DATE_GMT', 'water_level']]
df_non_zero = df_long[df_long['water_level'] != 0]
df_non_zero.set_index('DATE_GMT', inplace=True)
df_non_zero.index.freq='D'
# df_non_zero.plot(figsize=(12,6))
df_non_zero.tail(5)

,water_level
DATE_GMT,
10-23-2024,7.09
10-24-2024,6.96
10-27-2024,6.53
10-28-2024,6.30
10-31-2024,6.37


# Data V2

In [4]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import json
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import lxml

# Define the path to your Excel file
file_path = '../src/data/ManualData_Mainstream.xlsx'

# Load the data initially without parsing dates
data = pd.read_excel(
    file_path,
    sheet_name="in",
    skiprows=4,      
    header=[0, 1, 2]
)
# Fill NaN rainfall with 0
data.loc[:, (slice(None), slice(None), 'RF')] = data.loc[:, (slice(None), slice(None), 'RF')].fillna(0)


In [5]:
data.set_index('StaName', inplace=True)

# Filter for "Stung Treng" measurements
bassac_caktomuk = data.loc[:, pd.IndexSlice['Bassac Chaktomuk', 33401, :]]

# Reset index to bring 'Date' back as a column
bassac_caktomuk.reset_index(inplace=True)
bassac_caktomuk.columns = bassac_caktomuk.columns.droplevel([0, 1])
bassac_caktomuk.columns = ['Date', 'WL 7AM', 'WL 7PM', 'RF']
bassac_caktomuk['Date'] = bassac_caktomuk['Date'].apply(lambda x: x[0] if isinstance(x, tuple) else x)
bassac_caktomuk['Date'] = pd.to_datetime(bassac_caktomuk['Date'])
bassac_caktomuk.set_index('Date', inplace=True)


bassac_caktomuk['WL 7AM'] = bassac_caktomuk['WL 7AM'].interpolate(method='linear')
bassac_caktomuk['WL 7PM'] = bassac_caktomuk['WL 7PM'].interpolate(method='linear')


# Drop Columns
bassac_caktomuk = bassac_caktomuk.drop('WL 7PM', axis=1)
bassac_caktomuk = bassac_caktomuk.drop('RF', axis=1)

/tmp/ipykernel_30529/371176613.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bassac_caktomuk['Date'] = bassac_caktomuk['Date'].apply(lambda x: x[0] if isinstance(x, tuple) else x)
/tmp/ipykernel_30529/371176613.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bassac_caktomuk['Date'] = pd.to_datetime(bassac_caktomuk['Date'])
/tmp/ipykernel_30529/371176613.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

# Train

In [6]:
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, GRU
from tensorflow.keras.callbacks import EarlyStopping

# Define split ratios
train_ratio = 0.7
valid_ratio = 0.15  # proportion of the entire dataset for validation
test_ratio = 0.15  # proportion of the entire dataset for testing

# Calculate the split indices
train_size = int(len(bassac_caktomuk) * train_ratio)
valid_size = int(len(bassac_caktomuk) * valid_ratio)
test_size = len(bassac_caktomuk) - train_size - valid_size

# Split data into training, validation, and testing sets
train = bassac_caktomuk.iloc[:train_size]
valid = bassac_caktomuk.iloc[train_size:train_size + valid_size]
test = bassac_caktomuk.iloc[train_size + valid_size:]

# Scale the data
scaler = MinMaxScaler()
scaled_train = scaler.fit_transform(train)  # Fit on train and transform all sets
scaled_valid = scaler.transform(valid)
scaled_test = scaler.transform(test)

# Define look back and batch size
look_back = 10  # Number of previous time steps to consider for prediction
batch_size = 32  # Batch size

# Create TimeseriesGenerator for training, validation, and testing data
train_generator = TimeseriesGenerator(scaled_train, scaled_train, length=look_back, batch_size=batch_size)
valid_generator = TimeseriesGenerator(scaled_valid, scaled_valid, length=look_back, batch_size=batch_size)
test_generator = TimeseriesGenerator(scaled_test, scaled_test, length=look_back, batch_size=batch_size)

In [ ]:
# Build the model
model = Sequential()
model.add(LSTM(200, activation='relu', return_sequences=True, input_shape=(look_back, scaled_train.shape[1])))
model.add(LSTM(100, return_sequences=True))
model.add(GRU(50))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# Set up early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with validation data
history = model.fit(train_generator, 
                    validation_data=valid_generator, 
                    epochs=100, 
                    callbacks=[early_stop],
                    verbose=1)

2024-11-02 01:18:33.662011: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/home/vscode/.local/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100


/home/vscode/.local/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


142/142 ━━━━━━━━━━━━━━━━━━━━ 15s 60ms/step - loss: 0.0114 - val_loss: 0.0061
Epoch 2/100
142/142 ━━━━━━━━━━━━━━━━━━━━ 9s 55ms/step - loss: 0.0042 - val_loss: 0.0026
Epoch 3/100
142/142 ━━━━━━━━━━━━━━━━━━━━ 10s 54ms/step - loss: 0.0030 - val_loss: 0.0030
Epoch 4/100
142/142 ━━━━━━━━━━━━━━━━━━━━ 8s 54ms/step - loss: 0.0026 - val_loss: 0.0024
Epoch 5/100
142/142 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step - loss: 0.0024 - val_loss: 0.0025
Epoch 6/100
142/142 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step - loss: 0.0020 - val_loss: 0.0016
Epoch 7/100
142/142 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - loss: 0.0013 - val_loss: 0.0021
Epoch 8/100
142/142 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step - loss: 0.0015 - val_loss: 0.0011
Epoch 9/100
142/142 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step - loss: 0.0023 - val_loss: 0.0014
Epoch 10/100
142/142 ━━━━━━━━━━━━━━━━━━━━ 7s 47ms/step - loss: 0.0011 - val_loss: 8.5125e-04
Epoch 11/100
142/142 ━━━━━━━━━━━━━━━━━━━━ 8s 56ms/step - loss: 9.6642e-04 - val_loss: 7.2080e-04
Epoch 12/100
142/142 ━━━━━━━

In [3]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Evaluate the model on test data
loss = model.evaluate(test_generator, verbose=1)
print(f"Test Loss: {loss}")

# Make predictions on the test data
predictions = model.predict(test_generator)

# Inverse scale predictions to original values
predictions_inverse = scaler.inverse_transform(predictions)

# Prepare the test data corresponding to predictions for comparison
test_data = scaled_test[look_back:]  # Offset by 'look_back' to align with generator output
original_test_data_inverse = scaler.inverse_transform(test_data)

# Adjust test_dates to match the length of predictions
# Assuming 'df_non_zero' contains the full dataset with a DateTime index
test_dates = bassac_caktomuk.index[len(train) + len(valid) + look_back: len(train) + len(valid) + len(predictions) + look_back]

# Plot predicted vs original values with dates
plt.figure(figsize=(12, 6))
plt.plot(test_dates, original_test_data_inverse[:, 0], label='Original Values', color='blue')
plt.plot(test_dates, predictions_inverse[:, 0], label='Predicted Values', linestyle='dashed', color='orange')
plt.title('Predicted vs Original Values')
plt.xlabel('Date')
plt.ylabel('Water Level')
plt.xticks(rotation=45)
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=5))
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))

plt.legend()
plt.tight_layout()
plt.show()


NameError: name 'model' is not defined